In [122]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [123]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24"

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [124]:
# Find table element
table = soup.find("table", class_="Crom_table__p1iZz")

# Get table headers
headers = []
for i in range(len(table.thead.find_all('th'))):
    header = table.thead.find_all('th')[i].text
    headers.append(header)

headers

['#',
 'Player',
 'TEAM',
 'GP',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'EFF']

6

In [146]:
# Scrape data from all pages for current year

# Get the last page of stats for selected year
last_page = int(soup.find_all('select')[-1].contents[-1].text)
last_page
    
# Get data from table
data = []
for row in table.tbody.find_all('tr'):
    cols = row.find_all('td')
    
    if len(cols) == 0:
        cols.find_all('th')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
    
    # Gets all stats until the last page
    for x in range(last_page):
        browser.find_by_css('.Pagination_button__sqGoH')[1].click()

# Print a part of the array containing each row of data
data

AttributeError: 'str' object has no attribute 'text'

In [142]:
data

[['1',
  'Luka Doncic',
  'DAL',
  '10',
  '35.8',
  '32.6',
  '11.2',
  '21.3',
  '52.6',
  '4.3',
  '10.3',
  '41.7',
  '5.9',
  '7.5',
  '78.7',
  '0.6',
  '7.8',
  '8.4',
  '8.5',
  '1.3',
  '0.5',
  '4.2',
  '35.4'],
 ['2',
  'Joel Embiid',
  'PHI',
  '9',
  '33.8',
  '32.4',
  '11.0',
  '21.8',
  '50.5',
  '1.3',
  '3.7',
  '36.4',
  '9.1',
  '10.8',
  '84.5',
  '3.4',
  '8.2',
  '11.7',
  '5.7',
  '0.7',
  '2.1',
  '3.8',
  '36.3'],
 ['3',
  'Stephen Curry',
  'GSW',
  '11',
  '32.5',
  '30.7',
  '9.5',
  '19.5',
  '48.8',
  '5.6',
  '12.6',
  '44.6',
  '6.0',
  '6.5',
  '93.0',
  '0.9',
  '3.7',
  '4.6',
  '3.9',
  '1.0',
  '0.2',
  '3.7',
  '26.3'],
 ['4',
  'Kevin Durant',
  'PHX',
  '10',
  '36.7',
  '30.0',
  '10.1',
  '20.3',
  '49.8',
  '1.8',
  '4.2',
  '42.9',
  '8.0',
  '9.4',
  '85.1',
  '0.5',
  '6.6',
  '7.1',
  '4.6',
  '0.9',
  '1.2',
  '4.1',
  '28.1'],
 ['5',
  'Nikola Jokic',
  'DEN',
  '10',
  '34.6',
  '29.8',
  '12.0',
  '19.8',
  '60.6',
  '1.5',
  '4.5',
 

In [126]:
# Get current year of stats
select_year = soup.find_all("select")[0]
year_options = []
for i in range(1, len(select_year)):
    # Store each year in the years array
    year_options.append(select_year.contents[i].text)
    
# Scrape stats for remaining years from dropdown menu
for x in range(len(year_options)):
    current_year_stats = year_options[x]
    
    # Url for scraping the other years
    url = f"https://www.nba.com/stats/leaders?Season={current_year_stats}"
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    
    # Find table element
    table = soup.find("table", class_="Crom_table__p1iZz")

    # Get data from table
    data = []
    for row in table.tbody.find_all('tr'):
        cols = row.find_all('td')
        
        if len(cols) == 0:
            cols.find_all('th')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

        print(f"Gathering data for season: {current_year_stats}")

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Getting the headers from the data array
# It is important to remove them from the data array afterwards in order to be parsed correctly by Pandas
df = pd.DataFrame(data, columns=headers)
df


,#,Player,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,1,James Harden,HOU,68,36.5,34.3,9.9,22.3,44.4,4.4,...,11.8,86.5,1.0,5.5,6.6,7.5,1.8,0.9,4.5,32.6
1,2,Bradley Beal,WAS,57,36.0,30.5,10.4,22.9,45.5,3.0,...,8.0,84.2,0.9,3.3,4.2,6.1,1.2,0.4,3.4,25.4
2,3,Damian Lillard,POR,66,37.5,30.0,9.5,20.4,46.3,4.1,...,7.8,88.8,0.5,3.8,4.3,8.0,1.1,0.3,2.9,28.9
3,4,Trae Young,ATL,60,35.3,29.6,9.1,20.8,43.7,3.4,...,9.3,86.0,0.5,3.7,4.3,9.3,1.1,0.1,4.8,26.6
4,5,Giannis Antetokounmpo,MIL,63,30.4,29.5,10.9,19.7,55.3,1.4,...,10.0,63.3,2.2,11.4,13.6,5.6,1.0,1.0,3.7,34.6
5,6,Luka Doncic,DAL,61,33.6,28.8,9.5,20.6,46.3,2.8,...,9.2,75.8,1.3,8.1,9.4,8.8,1.0,0.2,4.3,30.8
6,7,Russell Westbrook,HOU,57,35.9,27.2,10.6,22.5,47.2,1.0,...,6.7,76.3,1.8,6.2,7.9,7.0,1.6,0.4,4.5,26.2
7,8,Kawhi Leonard,LAC,57,32.4,27.1,9.3,19.9,47.0,2.2,...,7.1,88.6,0.9,6.1,7.1,4.9,1.8,0.6,2.6,27.5
8,9,Devin Booker,PHX,70,35.9,26.6,9.0,18.3,48.9,2.0,...,7.3,91.9,0.4,3.8,4.2,6.5,0.7,0.3,3.8,24.6
9,10,Anthony Davis,LAL,62,34.4,26.1,8.9,17.7,50.3,1.2,...,8.5,84.6,2.3,7.0,9.3,3.2,1.5,2.3,2.5,29.8
